In [16]:
import os 
import pandas as pd 
from bs4 import BeautifulSoup
from io import StringIO

In [ ]:
%pip install pandas

In [ ]:
SCORE_DIR = "data/scores"

In [ ]:
box_scores = os.listdir(SCORE_DIR)

In [ ]:
box_scores

In [ ]:
len(box_scores)

In [ ]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [ ]:
box_scores

In [181]:

def parse_html(box_score):
    with open(box_score, encoding="utf-8") as f:  
        html = f.read()
    soup = BeautifulSoup(html, "html.parser")  
    [s.decompose() for s in soup.select("tr.over_header")] 
    return soup
    
    
    

In [182]:
def read_line_score(soup): #use pandas to process html table as dataframe
    html_string = str(soup)
    html_buffer = StringIO(html_string)
    line_score = pd.read_html(html_buffer, attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols


    line_score = line_score[["team", "total"]]
    
    return line_score

In [183]:

def read_stats(soup, team, stat):
    """Reads a stats table for a given team and stat type."""
    html_string = str(soup)
    html_buffer = StringIO(html_string) 
    df = pd.read_html(html_buffer, attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df  

In [184]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [ ]:
base_cols = None
games = []
box_score = box_scores[0]
for box_score in box_scores:
    soup = parse_html(box_score)
    #html_string = str(soup)
    #html_buffer = StringIO(html_string)
    #df = pd.read_html(html_buffer, attrs={"id": "line_score"})[0]
    #df
    #line_score = pd.read_html(html_buffer, attrs={"id": "line_score"})[0]
    #cols = list(line_score.columns)
    #cols[0] = "Team"
    #cols[-1] = "Total"
    #line_score.columns = cols
    
    #line_score
    line_score = read_line_score(soup)
    
    teams = list(line_score["team"])
    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()
        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"
        summary = pd.concat([totals,maxes])
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        summary = summary[base_cols]
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T
    game = pd.concat([summary, line_score], axis=1)
    game["home"] = [0,1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"
    full_game = pd.concat([game, game_opp], axis=1)
    full_game["seasons"] = read_season_info(soup)
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 9769


In [ ]:
#soup

In [ ]:
#teams

In [ ]:
#df

In [ ]:
#advanced

In [ ]:
#basic

In [ ]:
#total

In [ ]:
#maxes

In [ ]:
#totals

In [ ]:
#summary

In [ ]:
#game

In [ ]:
#game_opp

In [ ]:
#full_game

In [ ]:
#box_score

In [ ]:
games_df = pd.concat(games, ignore_index=True)

In [ ]:
games_df